In [1]:
import pandas as pd
import numpy as np

In [2]:
#Functions for cleaning the TLC yellow and green taxi datasets, before combining them
def preprocess(x):
    x = x[(x["total_amount"] > 2.5) & (x["total_amount"] < 50)]
    x = x[(x["passenger_count"] > 0) & (x["passenger_count"] < 7)]
    x = x[(x["trip_distance"] > 0) & (x["trip_distance"] < 20)]
    x = x[x["RatecodeID"] == 1]
    x = x[(x["payment_type"] == 1) | (x["payment_type"] == 2)]
    x = x[(x["PULocationID"]!= 264) & (x["PULocationID"]!= 265) & \
          (x["DOLocationID"]!= 264) & (x["DOLocationID"]!= 265)]
    
    
    x = x.astype({'tpep_pickup_datetime':'datetime64[ns]', 'tpep_dropoff_datetime':'datetime64[ns]', \
                 'PULocationID': 'category', 'DOLocationID': 'category', 'payment_type': 'category', \
                 'RatecodeID': 'category', 'VendorID': 'category', 'passenger_count': 'int32', \
                  'trip_distance':'float32', 'fare_amount':'float32', 'extra':'float32', 'mta_tax':'float32', \
                 'tip_amount':'float32', 'tolls_amount':'float32', 'improvement_surcharge':'float32', \
                 'total_amount':'float32'})
    
    x = x.rename(columns = {"tpep_pickup_datetime":"pickup_time", \
                            "tpep_dropoff_datetime":"dropoff_time"})
    
    
    return x

In [3]:
def preprocess_green(x):
    x = x.rename(columns = {"lpep_pickup_datetime":"tpep_pickup_datetime", \
                            "lpep_dropoff_datetime":"tpep_dropoff_datetime"})
    x = x.drop(["ehail_fee", "trip_type"], axis = 1)
    return x

In [4]:
#Now, engineer new features and conduct further filtering on those

In [5]:
def time(x):
    if x < 6 and x > 1:
        return 'Morning'
    if x < 18:
        return 'Day'
    if x < 22:
        return 'Evening'
    return 'Night'

In [6]:
def engineer(x, what_year):
    x['year'] = x['pickup_time'].dt.year.astype('category')
    x = x[x['year'] == what_year]
    
    
    x['month'] = x['pickup_time'].dt.month
    x = x[x['month'] < 7]
    x['month'] = x['month'].astype('category')
    
    x['hour'] = x['pickup_time'].dt.hour.astype('category')
    x['dow'] = x['pickup_time'].dt.dayofweek.astype('category')
    
    x['cost'] = x['total_amount'] - x['tip_amount']
    
    x = x[(x['cost'] > 2.5) & (x['cost'] < 50)]
    x['tod'] = np.vectorize(time)(x['hour'])
    x['tod'] = x['tod'].astype('category')
    
    x = x.drop(['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', \
            'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge'], axis = 1)
    return x

In [5]:
ydf1 = pd.read_csv("../raw_data/yellow_tripdata_2018-01.csv")
ydf2 = pd.read_csv("../raw_data/yellow_tripdata_2018-02.csv")
ydf3 = pd.read_csv("../raw_data/yellow_tripdata_2018-03.csv")
ydf4 = pd.read_csv("../raw_data/yellow_tripdata_2018-04.csv")
ydf5 = pd.read_csv("../raw_data/yellow_tripdata_2018-05.csv")
ydf6 = pd.read_csv("../raw_data/yellow_tripdata_2018-06.csv")

gdf1 = pd.read_csv("../raw_data/green_tripdata_2018-01.csv")
gdf2 = pd.read_csv("../raw_data/green_tripdata_2018-02.csv")
gdf3 = pd.read_csv("../raw_data/green_tripdata_2018-03.csv")
gdf4 = pd.read_csv("../raw_data/green_tripdata_2018-04.csv")
gdf5 = pd.read_csv("../raw_data/green_tripdata_2018-05.csv")
gdf6 = pd.read_csv("../raw_data/green_tripdata_2018-06.csv")

In [8]:
ydf1 = preprocess(ydf1)
ydf2 = preprocess(ydf2)
ydf3 = preprocess(ydf3)
ydf4 = preprocess(ydf4)
ydf5 = preprocess(ydf5)
ydf6 = preprocess(ydf6)

In [9]:
gdf1 = preprocess_green(gdf1)
gdf2 = preprocess_green(gdf2)
gdf3 = preprocess_green(gdf3)
gdf4 = preprocess_green(gdf4)
gdf5 = preprocess_green(gdf5)
gdf6 = preprocess_green(gdf6)

gdf1 = preprocess(gdf1)
gdf2 = preprocess(gdf2)
gdf3 = preprocess(gdf3)
gdf4 = preprocess(gdf4)
gdf5 = preprocess(gdf5)
gdf6 = preprocess(gdf6)

In [12]:
ydf1 = engineer(ydf1, 2018)
ydf2 = engineer(ydf2, 2018)
ydf3 = engineer(ydf3, 2018)
ydf4 = engineer(ydf4, 2018)
ydf5 = engineer(ydf5, 2018)
ydf6 = engineer(ydf6, 2018)

/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [13]:
gdf1 = engineer(gdf1, 2018)
gdf2 = engineer(gdf2, 2018)
gdf3 = engineer(gdf3, 2018)
gdf4 = engineer(gdf4, 2018)
gdf5 = engineer(gdf5, 2018)
gdf6 = engineer(gdf6, 2018)

In [6]:
total18 = pd.concat([ydf1, ydf2, ydf3, ydf4, ydf5, ydf6, gdf1, gdf2, gdf3, gdf4, gdf5, gdf6])

In [47]:
zones = pd.read_csv('../raw_data/zone_lookup.csv')

In [50]:
boroughs = zones[['LocationID', 'Borough']]

In [51]:
total18 = total18.merge(boroughs, how = 'left', left_on = 'PULocationID', right_on = 'LocationID') \
.rename({"Borough" : "PUBorough"}, axis = 1)

In [52]:
total18 = total18.merge(boroughs, how = 'left', left_on = 'DOLocationID', right_on = 'LocationID') \
.rename({"Borough" : "DOBorough"}, axis = 1)

In [54]:
total18 = total18.drop(['LocationID_x', 'LocationID_y'], axis = 1)

In [22]:
total18.to_pickle('../preprocessed_data/2018cleaned.pkl')

In [7]:
ydf1 = pd.read_csv("../raw_data/yellow_tripdata_2019-01.csv")
ydf2 = pd.read_csv("../raw_data/yellow_tripdata_2019-02.csv")
ydf3 = pd.read_csv("../raw_data/yellow_tripdata_2019-03.csv")
ydf4 = pd.read_csv("../raw_data/yellow_tripdata_2019-04.csv")
ydf5 = pd.read_csv("../raw_data/yellow_tripdata_2019-05.csv")
ydf6 = pd.read_csv("../raw_data/yellow_tripdata_2019-06.csv")

gdf1 = pd.read_csv("../raw_data/green_tripdata_2019-01.csv")
gdf2 = pd.read_csv("../raw_data/green_tripdata_2019-02.csv")
gdf3 = pd.read_csv("../raw_data/green_tripdata_2019-03.csv")
gdf4 = pd.read_csv("../raw_data/green_tripdata_2019-04.csv")
gdf5 = pd.read_csv("../raw_data/green_tripdata_2019-05.csv")
gdf6 = pd.read_csv("../raw_data/green_tripdata_2019-06.csv")

In [8]:
ydf1 = preprocess(ydf1)
ydf2 = preprocess(ydf2)
ydf3 = preprocess(ydf3)
ydf4 = preprocess(ydf4)
ydf5 = preprocess(ydf5)
ydf6 = preprocess(ydf6)

In [9]:
gdf1 = preprocess_green(gdf1)
gdf2 = preprocess_green(gdf2)
gdf3 = preprocess_green(gdf3)
gdf4 = preprocess_green(gdf4)
gdf5 = preprocess_green(gdf5)
df6 = preprocess_green(gdf6)

gdf1 = preprocess(gdf1)
gdf2 = preprocess(gdf2)
gdf3 = preprocess(gdf3)
gdf4 = preprocess(gdf4)
gdf5 = preprocess(gdf5)
gdf6 = preprocess(gdf6)

In [10]:
ydf1 = engineer(ydf1, 2019)
ydf2 = engineer(ydf2, 2019)
ydf3 = engineer(ydf3, 2019)
ydf4 = engineer(ydf4, 2019)
ydf5 = engineer(ydf5, 2019)
ydf6 = engineer(ydf6, 2019)

<ipython-input-6-c780c5c82625>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['month'] = x['pickup_time'].dt.month


In [11]:
gdf1 = engineer(gdf1, 2019)
gdf2 = engineer(gdf2, 2019)
gdf3 = engineer(gdf3, 2019)
gdf4 = engineer(gdf4, 2019)
gdf5 = engineer(gdf5, 2019)
gdf6 = engineer(gdf6, 2019)

<ipython-input-6-c780c5c82625>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['month'] = x['pickup_time'].dt.month


In [21]:
total19 = pd.concat([ydf1, ydf2, ydf3, ydf4, ydf5, ydf6, gdf1, gdf2, gdf3, gdf4, gdf5, gdf6])

In [14]:
zones = pd.read_csv('../raw_data/zone_lookup.csv')

In [15]:
boroughs = zones[['LocationID', 'Borough']]

In [22]:
total19 = total19.merge(boroughs, how = 'left', left_on = 'PULocationID', right_on = 'LocationID') \
.rename({"Borough" : "PUBorough"}, axis = 1)

In [23]:
total19 = total19.merge(boroughs, how = 'left', left_on = 'DOLocationID', right_on = 'LocationID') \
.rename({"Borough" : "DOBorough"}, axis = 1)

In [24]:
total19 = total19.drop(['LocationID_x', 'LocationID_y', 'congestion_surcharge'], axis = 1)

In [26]:
total19.to_pickle('../preprocessed_data/2018cleaned.pkl')